# Node.js Streams homework

### Reference
* Short intro https://nodejs.dev/learn/nodejs-streams
* Node.js docs https://nodejs.org/api/stream.html

### Warning

If you encounter ```SyntaxError: Identifier <some-variable-name> has already been declared```
go to Kernel -> Restart in order to clear all declared variables.

### Task 1
You will have to apply the concepts presented at class in order to convert an audio file from a format to another without reading it entirely into memory.

This process is generically called transcoding, although sometimes transmuxing is a more accurate term. You will understand the difference after you solve task 2.

Since an image is worth a thousand words, here is what you'll actually do (in a streamed fashion):

![title](../img/transcoding.png)

### Solution

Please, don't change anything in the ```DON'T CHANGE``` sections of the code.

You can test that the conversion is successfully done by accessing http://localhost:5050/, where you have an audio player.

I suggest opening the url in an incognito session to prevent browser caching.

In [1]:
'use strict';

///////////// DON'T CHANGE /////////////
// check docs for https://www.npmjs.com/package/prism-media
const prism = require('prism-media');
///////////////////////////////////////

// You can add your dependencies here. 
// Note that you should use only Node.js builtin modules.
// aka no third-party dependencies allowed, like the one above.
const fs = require('fs');

///////////// DON'T CHANGE /////////////
const SOURCE_FILE = '../data/weekend.mp3';
const DESTINATION_FILE = '../www/static/audio.ogg';

const OUT_MUXER = 'ogg';
const OUT_SAMPLE_RATE = '48000';
const OUT_NUM_CHANNELS = '2';

const transcoder = new prism.FFmpeg({
  args: [
    '-analyzeduration', '0',
    '-loglevel', '0',
    '-f', OUT_MUXER,
    '-ar', OUT_SAMPLE_RATE,
    '-ac', OUT_NUM_CHANNELS,
  ],
});
///////////////////////////////////////

// You can add your logic here.
// Note that you should use only Node.js Stream APIs.
// aka reading of the whole file in memory and afterwards converting it is not scored.
const readableStream = fs.createReadStream(SOURCE_FILE);
const writebleStream = fs.createWriteStream(DESTINATION_FILE);

readableStream.pipe(transcoder).pipe(writebleStream);




WriteStream {
  fd: null,
  path: '../www/static/audio.ogg',
  flags: 'w',
  mode: 438,
  start: undefined,
  pos: undefined,
  bytesWritten: 0,
  _writableState: WritableState {
    objectMode: false,
    highWaterMark: 16384,
    finalCalled: false,
    needDrain: false,
    ending: false,
    ended: false,
    finished: false,
    destroyed: false,
    decodeStrings: true,
    defaultEncoding: 'utf8',
    length: 0,
    writing: false,
    corked: 0,
    sync: true,
    bufferProcessing: false,
    onwrite: [Function: bound onwrite],
    writecb: null,
    writelen: 0,
    afterWriteTickInfo: null,
    buffered: [],
    bufferedIndex: 0,
    allBuffers: true,
    allNoop: true,
    pendingcb: 0,
    constructed: false,
    prefinished: false,
    errorEmitted: false,
    emitClose: true,
    autoDestroy: true,
    errored: null,
    closed: false,
    closeEmitted: false,
    [Symbol(kOnFinished)]: []
  },
  _events: [Object: null prototype] {
    unpipe: [Function: onunpipe],
    e

### Task 2

Use the following code to find out what codec is used by the source file and by the destination file, respectively. You should write them down along with the code changes you've made in order to use the cell given below.

Also answer to the following questions:
1. What's the difference between a container and a codec?
2. What format do you think is more appropriate for live streaming conferences? (the one used by the source file, or the one used by the destination file)

In [1]:
const { spawn } = require('child_process');
const ffprobe = require('ffprobe-static');

const SOURCE_FILE = '../data/weekend.mp3';
const DESTINATION_FILE = '../www/static/audio.ogg';

// process.argv = ['node', 'probe.js', DESTINATION_FILE];
process.argv = ['node', 'probe.js', SOURCE_FILE];

if (process.argv.length < 3) {
    console.error('Command format: node probe.js <path-to-media-file>');
}
const command = ffprobe.path;
const args = [
    '-v', 'error',
    '-select_streams', 'a:0',
    '-show_entries', 'stream=codec_long_name',
    '-of', 'default=noprint_wrappers=1:nokey=1',
    process.argv[2]
];

const subprocess = spawn(command, args);

subprocess.stdout.on('data', (data) => {
  console.log(`stdout: ${data}`);
});

subprocess.stderr.on('data', (data) => {
  console.error(`stderr: ${data}`);
});

subprocess.on('exit', (code, signal) => {

    if (code !== null) {

        if (code === 0) {
            console.log('Process exit with succes');
            return;
        }

        console.error(`Process exit code ${code}`);
        return;
    }

    if (signal !== null) {
        console.error(`Process killed with signal ${signal}`);
    }
});

<ref *1> ChildProcess {
  _events: [Object: null prototype] { exit: [Function (anonymous)] },
  _eventsCount: 1,
  _maxListeners: undefined,
  _closesNeeded: 3,
  _closesGot: 0,
  connected: false,
  signalCode: null,
  exitCode: null,
  killed: false,
  spawnfile: '/usr/project/hw/node_modules/ffprobe-static/bin/linux/x64/ffprobe',
  _handle: Process {
    onexit: [Function (anonymous)],
    pid: 115,
    [Symbol(owner_symbol)]: [Circular *1]
  },
  spawnargs: [
    '/usr/project/hw/node_modules/ffprobe-static/bin/linux/x64/ffprobe',
    '-v',
    'error',
    '-select_streams',
    'a:0',
    '-show_entries',
    'stream=codec_long_name',
    '-of',
    'default=noprint_wrappers=1:nokey=1',
    '../data/weekend.mp3'
  ],
  pid: 115,
  stdin: <ref *2> Socket {
    connecting: false,
    _hadError: false,
    _parent: null,
    _host: null,
    _readableState: ReadableState {
      objectMode: false,
      highWaterMark: 16384,
      buffer: BufferList { head: null, tail: null, length:

stdout: MP3 (MPEG audio layer 3)

Process exit with succes


In [ ]:
I modified the code by setting the arguments properly. If we set the media file to the .mp3 file, the output will be MP3 (MPEG audio layer 3)
and for the .ogg it will be Vorbis.

In [ ]:
A codec is used for faster transmisions or to make a file smaller by using two components, an encoder to compress the files and a decoder to decompress.
On the other hand, a container is used to package parts of a file. So, the codec is the software used for compression while the container is the final product, the delivered package.

In [ ]:
I think that the ogg format is better for live streaming conferences because it is superior in sound quality and it has smaller size.